### Homework 5: Question search engine

Remeber week01 where you used GloVe embeddings to find related questions? That was.. cute, but far from state of the art. It's time to really solve this task using context-aware embeddings.

__Warning:__ this task assumes you have seen `seminar.ipynb`!

In [1]:
%pip install --upgrade transformers datasets accelerate deepspeed
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
import datasets

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


### Load data and model

In [2]:
qqp = datasets.load_dataset('SetFit/qqp')
print('\n')
print("Sample[0]:", qqp['train'][0])
print("Sample[3]:", qqp['train'][3])

Repo card metadata block was not found. Setting CardData to empty.




Sample[0]: {'text1': 'How is the life of a math student? Could you describe your own experiences?', 'text2': 'Which level of prepration is enough for the exam jlpt5?', 'label': 0, 'idx': 0, 'label_text': 'not duplicate'}
Sample[3]: {'text1': 'What can one do after MBBS?', 'text2': 'What do i do after my MBBS ?', 'label': 1, 'idx': 3, 'label_text': 'duplicate'}


In [3]:
model_name = "gchhablani/bert-base-cased-finetuned-qqp"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

2024-10-27 13:32:19.039903: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-27 13:32:24.230871: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by call

### Tokenize the data

In [4]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

In [5]:
print(repr(qqp_preprocessed['train'][0]['input_ids'])[:100], "...")

[101, 1731, 1110, 1103, 1297, 1104, 170, 12523, 2377, 136, 7426, 1128, 5594, 1240, 1319, 5758, 136,  ...


### Task 1: evaluation (1 point)

We randomly chose a model trained on QQP - but is it any good?

One way to measure this is with validation accuracy - which is what you will implement next.

Here's the interface to help you do that:

In [17]:
#qqp_preprocessed['validation']['label']

In [50]:
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=1, shuffle=False, collate_fn=transformers.default_data_collator
)

In [8]:
for batch in val_loader:
     break  # here be your training code
print("Sample batch:", batch)

with torch.no_grad():
  predicted = model(
      input_ids=batch['input_ids'],
      attention_mask=batch['attention_mask'],
      token_type_ids=batch['token_type_ids']
  )

print('\nPrediction (probs):', torch.softmax(predicted.logits, dim=1).data.numpy())

Sample batch: {'labels': tensor([0]), 'idx': tensor([0]), 'input_ids': tensor([[  101,  2009,  1132,  2170,   118,  4038,  1177,  2712,   136,   102,
          2009,  1132,  1117, 10224,  4724,  1177,  2712,   136,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,   

__Your task__ is to measure the validation accuracy of your model.
Doing so naively may take several hours. Please make sure you use the following optimizations:

- run the model on GPU with no_grad
- using batch size larger than 1
- use optimize data loader with num_workers > 1
- (optional) use [mixed precision](https://pytorch.org/docs/stable/notes/amp_examples.html)


In [22]:
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score


val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=20, shuffle=False, collate_fn=transformers.default_data_collator, num_workers=4
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

predicts = []

model.eval()

with torch.no_grad():
    for batch in tqdm(val_loader):
        predicted = model(
            input_ids=batch['input_ids'].to(device),
            attention_mask=batch['attention_mask'].to(device),
            token_type_ids=batch['token_type_ids'].to(device)
        )
        
        probs = torch.softmax(predicted.logits, dim=1).cpu().data.numpy()
        preds = np.argmax(probs, axis=1)
        predicts.extend(preds)
        

accuracy = accuracy_score(qqp_preprocessed['validation']['label'], predicts)


  0%|          | 0/2022 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(tr

In [23]:
accuracy

0.9083848627256987

In [24]:
assert 0.9 < accuracy < 0.91

### Task 2: train the model (4 points)

For this task, you have two options:

__Option A:__ fine-tune your own model. You are free to choose any model __except for the original BERT.__ We recommend [DeBERTa-v3](https://huggingface.co/microsoft/deberta-v3-base). Better yet, choose the best model based on public benchmarks (e.g. [GLUE](https://gluebenchmark.com/)).

You can write the training code manually or use transformers.Trainer (see [this example](https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification)). Please make sure that your model's accuracy is at least __comparable__ with the above example for BERT.


__Option B:__ compare at least 3 pre-finetuned models (in addition to the above BERT model). For each model, report (1) its accuracy, (2) its speed, measured in samples per second in your hardware setup and (3) its size in megabytes. Please take care to compare models in equal setting, e.g. same CPU / GPU. Compile your results into a table and write a short (~half-page on top of a table) report, summarizing your findings.

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = transformers.AutoTokenizer.from_pretrained('microsoft/deberta-base')
model = transformers.AutoModelForSequenceClassification.from_pretrained('microsoft/deberta-base', num_labels=2).to(device)

2024-10-27 18:39:54.275067: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-27 18:39:57.798797: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by call

In [4]:
#for param in model.base_model.parameters():
#    param.requires_grad = False

In [5]:
#def tokenize_function(examples):
#    return tokenizer(examples["text1"], examples["text2"], padding="max_length", truncation=True)


#tokenized_datasets = qqp.map(tokenize_function, batched=True)

In [6]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

tokenized_datasets = qqp.map(preprocess_function, batched=True)

Map: 100%|██████████| 390965/390965 [00:32<00:00, 12160.84 examples/s]


In [7]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    #probs = torch.softmax(logits, dim=1).cpu().data.numpy()
    #preds = np.argmax(probs, axis=1)
    preds = np.argmax(logits, axis=1)
    return metric.compute(predictions=preds, references=labels)

In [8]:
training_args = transformers.TrainingArguments(
    output_dir='./output_dir',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    gradient_accumulation_steps=4
)

/home/jupyter/.local/lib/python3.10/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
data_collator = transformers.DataCollatorWithPadding(tokenizer=tokenizer)

In [10]:
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

[2024-10-27 18:42:05,977] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


In [11]:
trainer.train()

  3%|▎         | 500/17055 [03:34<1:57:18,  2.35it/s]

{'loss': 1.788, 'grad_norm': 21.204341888427734, 'learning_rate': 1.9413661682790972e-05, 'epoch': 0.09}


  6%|▌         | 1000/17055 [07:07<1:54:00,  2.35it/s]

{'loss': 1.3782, 'grad_norm': 15.398720741271973, 'learning_rate': 1.8827323365581942e-05, 'epoch': 0.18}


  9%|▉         | 1500/17055 [10:39<1:50:22,  2.35it/s]

...

100%|█████████▉| 2522/2527 [01:35<00:00, 26.55it/s]
                                                       
100%|██████████| 2527/2527 [01:35<00:00, 26.54it/s]
                                                   

{'eval_loss': 0.24340957403182983, 'eval_accuracy': 0.9113034875092753, 'eval_runtime': 95.4144, 'eval_samples_per_second': 423.73, 'eval_steps_per_second': 26.484, 'epoch': 3.0}


100%|██████████| 17055/17055 [2:08:38<00:00,  2.21it/s]

{'train_runtime': 7718.1709, 'train_samples_per_second': 141.424, 'train_steps_per_second': 2.21, 'train_loss': 0.8648181144738959, 'epoch': 3.0}


TrainOutput(global_step=17055, training_loss=0.8648181144738959, metrics={'train_runtime': 7718.1709, 'train_samples_per_second': 141.424, 'train_steps_per_second': 2.21, 'total_flos': 8.3662819890432e+16, 'train_loss': 0.8648181144738959, 'epoch': 2.9998680796798736})

In [14]:
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score

val_set = tokenized_datasets['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=20, shuffle=False, collate_fn=transformers.default_data_collator, num_workers=4
)

predicts = []

model.eval()

with torch.no_grad():
    for batch in tqdm(val_loader):
        predicted = model(
            input_ids=batch['input_ids'].to(device),
            attention_mask=batch['attention_mask'].to(device),
            token_type_ids=batch['token_type_ids'].to(device)
        )
        
        probs = torch.softmax(predicted.logits, dim=1).cpu().data.numpy()
        preds = np.argmax(probs, axis=1)
        predicts.extend(preds)
        

accuracy = accuracy_score(tokenized_datasets['validation']['label'], predicts)
print(accuracy)

  0%|          | 0/2022 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(tr

0.9113034875092753


### Task 3: try the full pipeline (1 point)

Finally, it is time to use your model to find duplicate questions.
Please implement a function that takes a question and finds top-5 potential duplicates in the training set. For now, it is fine if your function is slow, as long as it yields correct results.

Showcase how your function works with at least 5 examples.

In [120]:
def find_duplicate(question):
    with torch.no_grad():
        probas = []
        for text1 in tqdm(qqp['validation']['text1'][:1000]):
            ids = tokenizer(question, text1, padding='max_length', max_length=MAX_LENGTH, truncation=True)['input_ids']
            predicted = model(
                torch.as_tensor([ids]).to(device)
            )
            probs = torch.softmax(predicted.logits, dim=1).cpu().data.numpy()
            probas.extend(probs)
        np_probas = np.array(probas)
        duplicate_id = np.argmax(np_probas[:, 1])
        return qqp['validation']['text1'][duplicate_id]

In [98]:
find_duplicate('Will backwards time travel ever be possible?')

100%|██████████| 40430/40430 [10:44<00:00, 62.74it/s]


'Is time travel possible then after how long time?'

In [121]:
find_duplicate('Is African-Americans people beautiful?')

100%|██████████| 1000/1000 [00:15<00:00, 63.95it/s]


'Why are African-Americans so beautiful?'

In [122]:
find_duplicate('Why are people so obsessed with having a girlfriend/boyfriend?')

100%|██████████| 1000/1000 [00:15<00:00, 63.84it/s]


'Why are people so obsessed with having a girlfriend/boyfriend?'

In [123]:
find_duplicate('Why life is complicated?')

100%|██████████| 1000/1000 [00:15<00:00, 63.92it/s]


'Why is my life getting so complicated?'

In [124]:
find_duplicate('What are the best things to do in Hong Kong?')

100%|██████████| 1000/1000 [00:15<00:00, 64.06it/s]


'What are the best things to do in Hong Kong?'

__Bonus:__ for bonus points, try to find a way to run the function faster than just passing over all questions in a loop. For isntance, you can form a short-list of potential candidates using a cheaper method, and then run your tranformer on that short list. If you opted for this solution, please keep both the original implementation and the optimized one - and explain briefly what is the difference there.